In [18]:
from transformers import AutoModel, AutoTokenizer
import torch

model_name = "KBLab/bert-base-swedish-cased-squad-experimental"

In [21]:
from transformers import pipeline

nlp = pipeline("question-answering", model=model_name, tokenizer=model_name)

doc = "Utställningen pågår till och med 31 december 2019."
q = "Hur länge pågår utställningen?"

nlp({ 
    'question': q,
    'context': doc
})


{'score': 0.3919380307197571,
 'start': 33,
 'end': 49,
 'answer': '31 december 2019'}